In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import getpass
import pyodbc
import datetime
import matplotlib
import matplotlib.pyplot as plt
from mainQuery import query
from sklearn.utils import check_consistent_length, column_or_1d, check_array
import Buckets as bc
import woe
import scoringfunctions as sf
import bsfunctions as bs
import time
from scipy import stats

from sklearn.utils.fixes import isclose
from sklearn import metrics as met
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cross_validation import KFold
from pylab import rcParams
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_selection import RFE
rcParams['figure.figsize'] = 7, 7
pd.options.mode.chained_assignment = None

In [2]:
import imp
woe = imp.reload(woe)
sf = imp.reload(sf)
bs = imp.reload(bs)

In [3]:
connectString = 'DRIVER={SQL Server};SERVER=dwh.int.revoplus.ru;DATABASE=Revo_DW;UID=e.migaev;PWD='
pswd = getpass.getpass('DWH_Password: ')
connectString = connectString + pswd
cnxn = pyodbc.connect(connectString)
pswd = connectString = ''

DWH_Password: ········


df = pd.read_sql(query,cnxn)
df.to_csv('sourceSmaple.csv',index = False)

In [3]:
source = pd.read_csv('sourceSmaple.csv',encoding='cp1251')
bFlags = pd.read_csv('C:/YandexDisk/Work/Useful/BadFlags/badFlags.csv')
SourceBadFlags = pd.merge(source,bFlags,how='left')

C:\Users\e.migaev\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
lastDate = SourceBadFlags[(SourceBadFlags.badMob3.notnull())].sort_values('ApplicationDate',
                            ascending=False).reset_index().loc[int(0.01*len(SourceBadFlags)),'ApplicationDate']
lastDate = pd.to_datetime(lastDate)

In [5]:
dfVar = bs.mainTransform(SourceBadFlags)
dfVar = dfVar.drop(dfVar[(dfVar.badMob3.notnull())&(dfVar.badFpd.isnull())].index)

# Trader choose

In [6]:
kari = [6,25,44]
dm = [12,26]
allTrader = list(dfVar.traderKey.unique())

dfMob3Kari, dfFPDKari, dfOotKari, dfDeclKari = bs.chooseTrader(dfVar, kari,lastDate)
dfMob3dm, dfFPDdm, dfOotdm, dfDeclDM = bs.chooseTrader(dfVar, dm, lastDate)
dfMob3All,dfFPDAll, dfOotAll, dfDeclAll = bs.chooseTrader(dfVar,allTrader, lastDate)

In [9]:
dfMob3All.to_csv('AllMob3/src2.csv',index=False)

# WOE creation

In [371]:
begin_time = time.time()

dfPreWoeMob3All, clustVarsInfoMob3All = sf.continuousVariables(dfMob3All,50)
dfPreWoeFPDAll, clustVarsInfoFPDAll = sf.continuousVariables(dfFPDAll,50)
print('Mob3all:')
dfPostWoeMob3All, woeVarsInfoMob3All = sf.woeVariables(dfPreWoeMob3All,'badMob3')
print('FPDall:')
dfPostWoeFPDAll, woeVarsInfoFPDAll = sf.woeVariables(dfPreWoeFPDAll,'badFpd')

dfPreWoeMob3Kari, clustVarsInfoMob3Kari = sf.continuousVariables(dfMob3Kari,50)
dfPreWoeFPDKari, clustVarsInfoFPDKari = sf.continuousVariables(dfFPDKari,50)
print('MOb3Kari:')
dfPostWoeMob3Kari, woeVarsInfoMob3Kari = sf.woeVariables(dfPreWoeMob3Kari,'badMob3')
print('FPDKari:')
dfPostWoeFPDKari, woeVarsInfoFPDKari = sf.woeVariables(dfPreWoeFPDKari,'badFpd')

dfPreWoeMob3dm, clustVarsInfoMob3dm = sf.continuousVariables(dfMob3dm,50)
dfPreWoeFPDdm, clustVarsInfoFPDdm = sf.continuousVariables(dfFPDdm,50)
print('Mob3dm:')
dfPostWoeMob3dm, woeVarsInfoMob3dm = sf.woeVariables(dfPreWoeMob3dm,'badMob3')
print('FPDdm:')
dfPostWoeFPDdm, woeVarsInfoFPDdm = sf.woeVariables(dfPreWoeFPDdm,'badFpd')


end_time = time.time()

Mob3all:
Progress: 3.6%, 7.1%, 10.7%, 14.3%, 17.9%, 21.4%, 25.0%, 28.6%, 32.1%, 35.7%, 39.3%, 42.9%, 46.4%, 50.0%, 53.6%, 57.1%, 60.7%, 64.3%, 67.9%, 71.4%, 75.0%, 78.6%, 82.1%, 85.7%, 89.3%, 92.9%, 96.4%, 100.0%, 
FPDall:
Progress: 3.6%, 7.1%, 10.7%, 14.3%, 17.9%, 21.4%, 25.0%, 28.6%, 32.1%, 35.7%, 39.3%, 42.9%, 46.4%, 50.0%, 53.6%, 57.1%, 60.7%, 64.3%, 67.9%, 71.4%, 75.0%, 78.6%, 82.1%, 85.7%, 89.3%, 92.9%, 96.4%, 100.0%, 
MOb3Kari:
Progress: 3.6%, 7.1%, 10.7%, 14.3%, 17.9%, 21.4%, 25.0%, 28.6%, 32.1%, 35.7%, 39.3%, 42.9%, 46.4%, 50.0%, 53.6%, 57.1%, 60.7%, 64.3%, 67.9%, 71.4%, 75.0%, 78.6%, 82.1%, 85.7%, 89.3%, 92.9%, 96.4%, 100.0%, 
FPDKari:
Progress: 3.6%, 7.1%, 10.7%, 14.3%, 17.9%, 21.4%, 25.0%, 28.6%, 32.1%, 35.7%, 39.3%, 42.9%, 46.4%, 50.0%, 53.6%, 57.1%, 60.7%, 64.3%, 67.9%, 71.4%, 75.0%, 78.6%, 82.1%, 85.7%, 89.3%, 92.9%, 96.4%, 100.0%, 
Mob3dm:
Progress: 3.6%, 7.1%, 10.7%, 14.3%, 17.9%, 21.4%, 25.0%, 28.6%, 32.1%, 35.7%, 39.3%, 42.9%, 46.4%, 50.0%, 53.6%, 57.1%, 60.7%, 64.3%

In [60]:
dfDeclAll.to_csv('AllMob3/decl.csv',index=False)

dfDeclKari.to_csv('KariMob3/decl.csv',index=False)

dfDeclKari.to_csv('KariFpd/decl.csv',index=False)

dfDeclDM.to_csv('DMMob3/decl.csv',index=False)

dfDeclDM.to_csv('DMFpd/decl.csv',index=False)

In [363]:
dfMob3All.to_csv('AllMob3/src.csv',index=False)
dfOotAll.to_csv('AllMob3/oot.csv',index=False)
dfPreWoeMob3All.to_csv('AllMob3/preWoe.csv',index=False)
clustVarsInfoMob3All.to_csv('AllMob3/clustInfo.csv',index=False)
dfPostWoeMob3All.to_csv('AllMob3/postWoe.csv',index=False)
woeVarsInfoMob3All.to_csv('AllMob3/woeInfo.csv',index=False)

dfMob3Kari.to_csv('KariMob3/src.csv',index=False)
dfOotKari.to_csv('KariMob3/oot.csv',index=False)
dfPreWoeMob3Kari.to_csv('KariMob3/preWoe.csv',index=False)
clustVarsInfoMob3Kari.to_csv('KariMob3/clustInfo.csv',index=False)
dfPostWoeMob3Kari.to_csv('KariMob3/postWoe.csv',index=False)
woeVarsInfoMob3Kari.to_csv('KariMob3/woeInfo.csv',index=False)

dfFPDKari.to_csv('KariFpd/src.csv',index=False)
dfOotKari.to_csv('KariFpd/oot.csv',index=False)
dfPreWoeFPDKari.to_csv('KariFpd/preWoe.csv',index=False)
clustVarsInfoFPDKari.to_csv('KariFpd/clustInfo.csv',index=False)
dfPostWoeFPDKari.to_csv('KariFpd/postWoe.csv',index=False)
woeVarsInfoFPDKari.to_csv('KariFpd/woeInfo.csv',index=False)

dfMob3dm.to_csv('DMMob3/src.csv',index=False)
dfOotdm.to_csv('DMMob3/oot.csv',index=False)
dfPreWoeMob3dm.to_csv('DMMob3/preWoe.csv',index=False)
clustVarsInfoMob3dm.to_csv('DMMob3/clustInfo.csv',index=False)
dfPostWoeMob3dm.to_csv('DMMob3/postWoe.csv',index=False)
woeVarsInfoMob3dm.to_csv('DMMob3/woeInfo.csv',index=False)

dfFPDdm.to_csv('DMFpd/src.csv',index=False)
dfOotdm.to_csv('DMFpd/oot.csv',index=False)
dfPreWoeFPDdm.to_csv('DMFpd/preWoe.csv',index=False)
clustVarsInfoFPDdm.to_csv('DMFpd/clustInfo.csv',index=False)
dfPostWoeFPDdm.to_csv('DMFpd/postWoe.csv',index=False)
woeVarsInfoFPDdm.to_csv('DMFpd/woeInfo.csv',index=False)

In [128]:
(end_time - begin_time) / 60

24.97419562737147

In [119]:
(end_time - begin_time) / 60

27.885725875695545